### transposon_mapping.py calling 
### clean_bedwigfiles.py (to clean the bed and wig files).

### transposonread_profileplot_genome.py (to check the insertion and read distribution throughout the genome).

### transposonread_profileplot.py (to check the insertions and read distribution per chromosome in more detail).

### scatterplot_genes.py (to check the distribution for the number of insertions per gene and per essential gene).

### volcanoplot.py (only when comparing multiple datasets with different genetic backgrounds to see which genes have a significant change in insertion and read counts).

In [18]:

import os, sys
import warnings
import timeit
import numpy as np
import pysam

#### Importing specific functions inside the python modules folder
dirname = os.path.dirname(os.path.abspath('__file__'))
sys.path.insert(1,os.path.join(dirname,'..','python_transposonmapping','python_modules'))
from chromosome_and_gene_positions import chromosomename_roman_to_arabic, gene_position
from gene_names import gene_aliases
from samflag import samflags

In [19]:
dirname = os.path.dirname(os.path.abspath('__file__'))
sys.path.insert(1,os.path.join(dirname,'..','python_transposonmapping'))
from transposonmapping_satay import transposonmapper


In [20]:
transposonmapper(bamfile=None) ## if it is run  in spyder it wont complain ....

KeyError: 17